In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
import nibabel as nib
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import scipy.io as sio

model_file = 'BLAHFCN_PATCH_A2B_SH1200_saved_model_split'
dirs = ['H','L','M','N','O']



In [2]:
from torch.utils.data import Dataset
from torchvision import transforms

class SHDataSet(Dataset):
    def __init__(self,X):
        
        self.X = X
        self.to_tensor = transforms.ToTensor()
        
    def __len__(self):
        return np.amax(X.shape)
    
    def __getitem__(self, i):

        vec_a = self.X[:,:,:,:,i]
        vec_a = np.transpose(vec_a, (3, 0, 1, 2))
        a = torch.Tensor(vec_a)
    
        return a

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.fc1 = nn.Linear(3 * 3 * 3 * 15, 600)
        self.cn1 = nn.Conv3d(15,128,kernel_size=(3,3,3),stride=1,padding=(1,1,1))
        #print(self.cn1.shape)
        #self.fc2 = nn.Linear(600,300)
        self.bn = nn.BatchNorm3d(128)
        self.fc2 = nn.Linear(128*3*3*3, 300)
        self.fc3 = nn.Linear(300,60)
        self.fc4 = nn.Linear(60,200)
        self.fc5 = nn.Linear(200,15)
        
    def forward(self, x):
        #print(x.shape)
        x = F.relu(self.cn1(x))
        #print(x.shape)
        dimensions = x.shape
        x = self.bn(x)
        x = x.view(dimensions[0], -1)
        #x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
        
net = Net()
print(net)  

Net(
  (cn1): Conv3d(15, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=3456, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=60, bias=True)
  (fc4): Linear(in_features=60, out_features=200, bias=True)
  (fc5): Linear(in_features=200, out_features=15, bias=True)
)


In [4]:
model = net.load_state_dict(torch.load(model_file))

In [6]:
for i in range(len(dirs)):

    apply_to_file = "/Users/kurtschilling/Data/harmonization/%s_A2B_Patches.mat" % (dirs[i])
    nii_file = "/Users/kurtschilling/Data/harmonization/CDMRI_Challenge_2018/Testing_Data/%s/prisma/st/norm_dwi_SHfitOrder4_EvenOdd2_SH3000.nii.gz" % (dirs[i])
    save_file = "BLAHFCN_PATCHES_s%s_A2B_SH1200.nii.gz" % (dirs[i])
    print(apply_to_file)
    print(nii_file)
    print(save_file)

    # load and reshape
    mat_contents = sio.loadmat(apply_to_file)
    X = mat_contents['input1200']
    print(X.shape)
    
    nifti_path = os.path.normpath(nii_file)
    img = nib.load(nifti_path)
    data = img.get_fdata()
    dims = data.shape
    print(dims)
    
    Y = np.zeros((dims[0],dims[1],dims[2],dims[3]))
    print(Y.shape)
    Y = np.reshape(Y,(dims[0]*dims[1]*dims[2],dims[3]))
    print(Y.shape)
    
    shset = SHDataSet(X)
    print(len(shset))

    from torch.utils.data import DataLoader
    test_loader = DataLoader(shset, batch_size=1)
    
    batch_idx = 0
    for batch_idx, (data) in enumerate(test_loader):
    
        data = Variable(data).float()
    
        net_out = net(data)
        b = net_out.detach().numpy()
        c = np.reshape(b,(1,15))
        #print(c.shape)
        #print(batch_idx)
        Y[batch_idx,:]=c;
    
    
        if batch_idx % 100000 == 0:
            print(batch_idx)
            #print(b)

    Y = np.reshape(Y,(dims[0],dims[1],dims[2],dims[3]))
    print(Y.shape)
    
    new_img = nib.Nifti1Image(Y,img.affine,img.header)
    nib.save(new_img,save_file)

/Users/kurtschilling/Data/harmonization/H_A2B_Patches.mat
/Users/kurtschilling/Data/harmonization/CDMRI_Challenge_2018/Testing_Data/H/prisma/st/norm_dwi_SHfitOrder4_EvenOdd2_SH3000.nii.gz
BLAHFCN_PATCHES_sH_A2B_SH1200.nii.gz
(3, 3, 3, 15, 552960)
(96, 96, 60, 15)
(96, 96, 60, 15)
(552960, 15)
552960
0
100000
200000
300000
400000
500000
(96, 96, 60, 15)
/Users/kurtschilling/Data/harmonization/L_A2B_Patches.mat
/Users/kurtschilling/Data/harmonization/CDMRI_Challenge_2018/Testing_Data/L/prisma/st/norm_dwi_SHfitOrder4_EvenOdd2_SH3000.nii.gz
BLAHFCN_PATCHES_sL_A2B_SH1200.nii.gz
(3, 3, 3, 15, 552960)
(96, 96, 60, 15)
(96, 96, 60, 15)
(552960, 15)
552960
0
100000
200000
300000
400000
500000
(96, 96, 60, 15)
/Users/kurtschilling/Data/harmonization/M_A2B_Patches.mat
/Users/kurtschilling/Data/harmonization/CDMRI_Challenge_2018/Testing_Data/M/prisma/st/norm_dwi_SHfitOrder4_EvenOdd2_SH3000.nii.gz
BLAHFCN_PATCHES_sM_A2B_SH1200.nii.gz
(3, 3, 3, 15, 552960)
(96, 96, 60, 15)
(96, 96, 60, 15)
(552960, 